<h1>Importes necesarios</h1>

In [2]:
'''!pip install scikit-learn
!pip install pandas
!pip install numpy
!pip install joblib
!pip install tensorflow
!pip install keras'''

'!pip install scikit-learn\n!pip install pandas\n!pip install numpy\n!pip install joblib\n!pip install tensorflow\n!pip install keras'

In [3]:
import pandas as pd
import numpy as np
import time
from sklearn.decomposition import PCA
from mlxtend.frequent_patterns import apriori, association_rules
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, f1_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Input
import joblib
import os
import random
import tensorflow as tf

<frozen importlib._bootstrap>:488: DeprecationWarning: Type google.protobuf.pyext._message.ScalarMapContainer uses PyType_Spec with a metaclass that has custom tp_new. This is deprecated and will no longer be allowed in Python 3.14.
<frozen importlib._bootstrap>:488: DeprecationWarning: Type google.protobuf.pyext._message.MessageMapContainer uses PyType_Spec with a metaclass that has custom tp_new. This is deprecated and will no longer be allowed in Python 3.14.


<h1>Pipeline de limpieza de datos y preparación</h1>

In [5]:
class Limpiar(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.map(lambda x: x.strip().strip("'").strip() if isinstance(x, str) else x).iloc[:, 1:]

class CodificarCategoricos(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.dummy_columns = None

    def fit(self, X, y=None):
        dummies = pd.get_dummies(X.iloc[:, 1:4])
        self.dummy_columns = dummies.columns
        return self

    def transform(self, X):
        dummies = pd.get_dummies(X.iloc[:, 1:4])
        dummies = dummies.reindex(columns=self.dummy_columns, fill_value=0)
        X.columns = X.columns.astype(str)
        drop = list(X.columns[1:4])
        y = None
        if X.shape[1] > 41:
            y = X.iloc[:, 41].map({'Normal': 0, 'Ataque': 1})
            drop.append(X.columns[41])
        X = X.drop(columns=drop).astype('float')
        X_encoded = pd.concat([X, dummies, y], axis=1)
        X_encoded.columns.values[-1] = 'y'
        return X_encoded

class ImputarNulos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.fillna(X.mean())

class Escalar(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.scaler = MinMaxScaler()
    
    def fit(self, X, y=None):
        self.scaler.fit(X)
        return self

    def transform(self, X):
        X_scaled = pd.DataFrame(self.scaler.transform(X), columns=X.columns, index=X.index)
        return X_scaled

class Reducir(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.pca = PCA(n_components=5)
        
    def fit(self, X, y=None):
        if X.columns[-1] == 'y':
            X = X.drop(columns=X.columns[-1]).astype('float')
        self.pca.fit(X)
        return self
        
    def transform(self, X):
        y = None
        if X.columns[-1] == 'y':
            y = X.iloc[:,-1]
            X = X.drop(columns=X.columns[-1]).astype('float')
        X_reduced = pd.DataFrame(self.pca.transform(X), index=X.index)
        return pd.concat([X_reduced, y], axis=1)

pipeline = Pipeline([
    ('limpiar', Limpiar()),
    ('codificar_categoricos', CodificarCategoricos()),
    ('imputar_nulos', ImputarNulos()),
    ('escalar', Escalar()),
    ('reducir', Reducir()),
])
joblib.dump(pipeline, "pipeline.pkl")

['pipeline.pkl']

<h1>Ejecutar el pipeline sobre el set de datos</h1>

In [7]:
inicio = time.time()
df = pd.read_csv("muestra_ataques.csv", header=None)
df = pipeline.fit_transform(df)
joblib.dump(pipeline, "pipeline.pkl")
datos = df.to_numpy()

y = datos[:,-1]
X = datos[:,:-1]

X, y = np.array(X, dtype='float64'), np.array(y, dtype='int')
print(f'Tiempo de ejecucion: {(time.time() - inicio) / 60 : .2f} minutos')

Tiempo de ejecucion:  0.04 minutos


<h1>Definir set de entrenamiento del 80% y set de prueba del 20%</h1>

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
modelos=[]

<h1>Entrenamiento de modelo de regresión logistica</h1>

In [11]:
inicio = time.time()
modelo = LogisticRegression(
    penalty='l2',
    C=1.0,
    solver='lbfgs',
    max_iter=1000,
    class_weight='balanced',
    random_state=123
)
modelo = modelo.fit(X_train, y_train)
modelos.append([modelo, 'lr'])
print(f'Tiempo de ejecucion: {(time.time() - inicio) / 60 : .2f} minutos')

Tiempo de ejecucion:  0.00 minutos


<h1>Búsqueda aleatoria de hiperparámetros para árbol de clasificación con validación cruzada de 4 carpetas usando concurrencia</h1>

In [13]:
inicio = time.time()
modelo = DecisionTreeClassifier(random_state=1234)
parametros = {'max_depth': [1,2,3,4,5,6,7,8,9,10,11,12,None],
              'criterion': ['gini','entropy'],
              'min_samples_split': [2,3,4,5,6,7,8],
              'min_samples_leaf': [1,2,3,4,5,6],
              'ccp_alpha': [0,0.0001,0.001,0.01,0.1,1]
             }
random_search = RandomizedSearchCV(modelo, parametros, cv=4, random_state=123, n_jobs=-1, n_iter=500)
random_search.fit(X_train, y_train)
print(f'Tiempo de ejecucion: {(time.time() - inicio) / 60 : .2f} minutos')

Tiempo de ejecucion:  0.31 minutos


<h1>Entrenamiento de modelo de árbol de decision con mejores hiperparámetros</h1>

In [15]:
inicio = time.time()
modelo = DecisionTreeClassifier(max_depth = random_search.best_params_['max_depth'],
                             criterion = random_search.best_params_['criterion'],
                             min_samples_split = random_search.best_params_['min_samples_split'],
                             min_samples_leaf = random_search.best_params_['min_samples_leaf'],
                             ccp_alpha =random_search.best_params_['ccp_alpha'])
modelo = modelo.fit(X_train, y_train)
modelos.append([modelo, 'clf'])
print(f'Tiempo de ejecucion: {(time.time() - inicio) / 60 : .2f} minutos')

Tiempo de ejecucion:  0.00 minutos


<h1>Búsqueda aleatoria de hiperparámetros para modelo K vecinos cercanos con validación cruzada de 4 carpetas sobre una parte del set de entrenamiento usando concurrencia</h1>

In [17]:
inicio = time.time()
modelo = KNeighborsClassifier()
parametros = {'n_neighbors': range(1,60),
              'p': [a/10 for a in range(10,30,5)],
              'weights': ['distance', 'uniform'],
             }
grid_search = GridSearchCV(modelo, parametros, cv=4, n_jobs=-1)
grid_search.fit(X_train, y_train)
print(f'Tiempo de ejecucion: {(time.time() - inicio) / 60 : .2f} minutos')

Tiempo de ejecucion:  2.84 minutos


<h1>Entrenamiento para modelo de K vecinos cercanos con los mejores hiperparámetros</h1>

In [19]:
inicio = time.time()
modelo = KNeighborsClassifier(n_neighbors=grid_search.best_params_['n_neighbors'],
              p=grid_search.best_params_['p'],
              weights=grid_search.best_params_['weights'])

modelo = modelo.fit(X_train, y_train)
modelos.append([modelo, 'knn'])
print(f'Tiempo de ejecucion: {(time.time() - inicio) / 60 : .2f} minutos')

Tiempo de ejecucion:  0.00 minutos


<h1>Búsqueda aleatoria de hiperparámetros para máquina de soporte vectorial con validación cruzada de 3 carpetas usando concurrencia</h1>

In [21]:
inicio = time.time()
parametros = {'coef0': [75,100,125],
              'degree': [2,3,4],
              'kernel': ['linear','poly','rbf']
             }
modelo = svm.SVC()
grid_search = GridSearchCV(modelo, parametros, cv=4, n_jobs=-1)
grid_search.fit(X_train, y_train)
print(f'Tiempo de ejecucion: {(time.time() - inicio) / 60 : .2f} minutos')

Tiempo de ejecucion:  0.94 minutos


<h1>Entrenamiento de modelo máquina de soporte vectorial con mejores hiperparámetros</h1>

In [23]:
inicio = time.time()
modelo = svm.SVC(coef0=grid_search.best_params_['coef0'],
              degree=grid_search.best_params_['degree'],
              kernel=grid_search.best_params_['kernel'])

modelo.fit(X_train, y_train)
modelos.append([modelo, 'svm'])
print(f'Tiempo de ejecucion: {(time.time() - inicio) / 60 : .2f} minutos')

Tiempo de ejecucion:  0.25 minutos


<h1>Entrenamiento de modelo de redes neuronales con parte del set de entrenamiento, con una capa de entrada, dos capas ocultas y una capa de salida con una funcion de activacion sigmoidal y una funcion de perdida de entropia binaria cruzada</h1>

In [25]:
inicio = time.time()
seed_value= 1234
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

modelo = Sequential()
modelo.add(Input(shape=(X_train.shape[1],)))
modelo.add(Dense(units=64, activation='relu'))
modelo.add(Dense(units=32, activation='relu'))
modelo.add(Dense(units=1, activation='sigmoid'))

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=1000,
    decay_rate=0.5)
opt = keras.optimizers.Adam(learning_rate=lr_schedule)
modelo.compile(loss='binary_crossentropy', optimizer=opt, metrics=['recall'])

modelo.fit(X_train, y_train, epochs=100,
          batch_size=1000,
          verbose = 0)

modelos.append([modelo, 'ann'])
print(f'Tiempo de ejecucion: {(time.time() - inicio) / 60 : .2f} minutos')

Tiempo de ejecucion:  0.49 minutos


<h1>Evaluación de los modelos con el set de entrenamiento según métricas exactitud, precisión, recall y f1-score</h1>

In [27]:
for modelo in modelos:
    print('\n-----------  '+modelo[1]+'  -----------')
    inicio = time.time()
    y_hat = (modelo[0].predict(X_test)>=0.5).astype(int)
    print(f'Tiempo de ejecucion: {(time.time() - inicio) / 60 : .2f} minutos')
    print('\nExactitud:', accuracy_score(y_test,y_hat))
    print('\nMatriz de confusion:', confusion_matrix(y_test, y_hat))
    print('\nReporte:\n', classification_report(y_test, y_hat))
    modelo.append(f1_score(y_test, y_hat))


-----------  lr  -----------
Tiempo de ejecucion:  0.00 minutos

Exactitud: 0.9477670666928979

Matriz de confusion: [[19018  1056]
 [    6   252]]

Reporte:
               precision    recall  f1-score   support

           0       1.00      0.95      0.97     20074
           1       0.19      0.98      0.32       258

    accuracy                           0.95     20332
   macro avg       0.60      0.96      0.65     20332
weighted avg       0.99      0.95      0.96     20332


-----------  clf  -----------
Tiempo de ejecucion:  0.00 minutos

Exactitud: 0.9982785756443046

Matriz de confusion: [[20067     7]
 [   28   230]]

Reporte:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     20074
           1       0.97      0.89      0.93       258

    accuracy                           1.00     20332
   macro avg       0.98      0.95      0.96     20332
weighted avg       1.00      1.00      1.00     20332


-----------  knn  ------

<h1>Despliegue y entrenamiento extra del mejor modelo según f1-score</h1>

In [29]:
modelos=sorted(modelos, key=lambda x: x[-1], reverse=True)
modelos[0][0] = modelos[0][0].fit(X_test, y_test)
if (modelos[0][1] == 'ann'):
    modelos[0][0].save("modelo.keras")
else:
    joblib.dump(modelos[0][0], "modelo.pkl")
print(f'Mejor modelo: {modelos[0][1]}')
print(f'F1-Score: {modelos[0][2]}')

Mejor modelo: knn
F1-Score: 0.97265625
